In [ ]:
# Helper packages
library(tidyverse)
library(rsample) # for data splitting
# Modeling packages
library(caret) # for logistic regression modeling
# Model interpretability packages
library(vip) # variable importance

In [12]:
attrition <- read_csv('data/attrition.csv')
attrition <- mutate(attrition, Attrition = recode(Attrition, `Yes` = 1, `No` =  0))


Parsed with column specification:
cols(
  .default = col_double(),
  Attrition = col_character(),
  BusinessTravel = col_character(),
  Department = col_character(),
  Education = col_character(),
  EducationField = col_character(),
  EnvironmentSatisfaction = col_character(),
  Gender = col_character(),
  JobInvolvement = col_character(),
  JobRole = col_character(),
  JobSatisfaction = col_character(),
  MaritalStatus = col_character(),
  OverTime = col_character(),
  PerformanceRating = col_character(),
  RelationshipSatisfaction = col_character(),
  WorkLifeBalance = col_character()
)

See spec(...) for full column specifications.



In [13]:
df <- attrition %>% mutate_if(is.ordered, factor, ordered = FALSE)
# Create training (70%) and test (30%) sets for the
# rsample::attrition data.

set.seed(123) # for reproducibility
churn_split <- initial_split(df, prop = .7, strata = "Attrition")
churn_train <- training(churn_split)
churn_test <- testing(churn_split)

In [16]:
# Set up binomial/logistic regressions

model1 <- glm(Attrition ~ MonthlyIncome, family = "binomial",
    data = churn_train)

tidy(model1)

term,estimate,std.error,statistic,p.value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),-0.9236247934,0.1550905456,-5.955391,2.594512e-09
MonthlyIncome,-0.0001302806,0.0000264429,-4.926867,8.355852e-07


In [17]:
model2 <- glm(Attrition ~ OverTime, family = "binomial",
    data = churn_train)

tidy(model2)

term,estimate,std.error,statistic,p.value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),-2.183802,0.1217895,-17.930951,6.761317e-72
OverTimeYes,1.406394,0.1757048,8.004299,1.201490e-15


In [18]:
# Translate the odds intro probabilities:

exp(coef(model1))
exp(coef(model2))

(Intercept) MonthlyIncome 
    0.3970771     0.9998697

(Intercept) OverTimeYes 
  0.1126126   4.0812121

the odds of an employee attriting in model1 increase multiplicatively by
1 for every one dollar increase in MonthlyIncome, whereas the odds of attriting
in model2 increase multiplicatively by 4.081 for employees that work OverTime
compared to those that do not.

In [19]:
# Get confidence intervals

confint(model1)
confint(model2)


Waiting for profiling to be done...



,2.5 %,97.5 %
(Intercept),-1.2267754960,-6.180062e-01
MonthlyIncome,-0.0001849796,-8.107634e-05


Waiting for profiling to be done...



,2.5 %,97.5 %
(Intercept),-2.430458,-1.952330
OverTimeYes,1.063246,1.752879


In [21]:
# Multiple logistic regression

model3 <- glm(
Attrition ~ MonthlyIncome + OverTime,
family = "binomial",
data = churn_train
)

tidy(model3)

term,estimate,std.error,statistic,p.value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),-1.4296820006,1.763808e-01,-8.105654,5.246248e-16
MonthlyIncome,-0.0001392218,2.703737e-05,-5.149235,2.615514e-07
OverTimeYes,1.4682839234,1.799996e-01,8.157150,3.430225e-16


In [22]:
set.seed(123)
cv_model1 <- train(
Attrition ~ MonthlyIncome,
data = churn_train,
method = "glm",
family = "binomial",
trControl = trainControl(method = "cv", number = 10)
)

Warning message in train.default(x, y, weights = w, ...):
"You are trying to do regression and your outcome only has two possible values Are you trying to do classification? If so, use a 2 level factor as your outcome column."


In [24]:
set.seed(123)
cv_model2 <- train(
Attrition ~ MonthlyIncome + OverTime,
data = churn_train,
    method = "glm",
family = "binomial",
trControl = trainControl(method = "cv", number = 10)
)

Warning message in train.default(x, y, weights = w, ...):
"You are trying to do regression and your outcome only has two possible values Are you trying to do classification? If so, use a 2 level factor as your outcome column."


In [25]:
set.seed(123)
cv_model3 <- train(
Attrition ~ .,
data = churn_train,
method = "glm",
family = "binomial",
trControl = trainControl(method = "cv", number = 10)
)

Warning message in train.default(x, y, weights = w, ...):
"You are trying to do regression and your outcome only has two possible values Are you trying to do classification? If so, use a 2 level factor as your outcome column."


In [26]:
# extract out of sample performance measures
summary(
resamples(
list(
model1 = cv_model1,
model2 = cv_model2,
model3 = cv_model3
)
)
)$statistics$Accuracy

NULL

In [28]:
# extract out of sample performance measures
summary(
resamples(
list(
model1 = cv_model1,
model2 = cv_model2,
model3 = cv_model3
)
)
)


Call:
summary.resamples(object = resamples(list(model1 = cv_model1, model2
 = cv_model2, model3 = cv_model3)))

Models: model1, model2, model3 
Number of resamples: 10 

MAE 
            Min.   1st Qu.    Median      Mean   3rd Qu.      Max. NA's
model1 0.2061740 0.2562651 0.2610893 0.2630697 0.2774781 0.3084759    0
model2 0.1868568 0.2287106 0.2477165 0.2423774 0.2606637 0.2846597    0
model3 0.1038123 0.1565367 0.1827211 0.1698659 0.1896536 0.1907121    0

RMSE 
            Min.   1st Qu.    Median      Mean   3rd Qu.      Max. NA's
model1 0.2669236 0.3571445 0.3593717 0.3605247 0.3842628 0.4279211    0
model2 0.2518288 0.3210373 0.3652731 0.3456446 0.3704597 0.4047592    0
model3 0.1962488 0.2791780 0.3105109 0.2996882 0.3356790 0.3443849    0

Rsquared 
              Min.    1st Qu.     Median       Mean    3rd Qu.      Max. NA's
model1 0.001104525 0.01613462 0.02659892 0.03746085 0.04143442 0.1034247    0
model2 0.002305286 0.07855734 0.11573486 0.12910230 0.13484299 0.3741519  

In [ ]:
# predict class
pred_class <- predict(cv_model3, churn_train)

In [38]:
pred_class

1            2            3            4            5            6 
8.987348e-01 3.937011e-03 5.276156e-01 4.287011e-02 1.354519e-01 2.795022e-02 
           7            8            9           10           11           12 
2.068691e-01 3.681014e-02 7.764212e-02 2.421463e-02 2.775524e-02 1.823948e-02 
          13           14           15           16           17           18 
8.957826e-01 1.213864e-01 3.750722e-02 2.283483e-01 1.451997e-03 4.725198e-01 
          19           20           21           22           23           24 
2.529628e-03 3.959023e-02 2.840482e-04 4.992797e-03 1.065429e-03 2.326219e-02 
          25           26           27           28           29           30 
1.579998e-01 8.461680e-02 4.864439e-04 8.366183e-01 7.695889e-02 6.840858e-03 
          31           32           33           34           35           36 
5.520702e-03 4.889055e-02 2.047172e-01 4.318050e-01 8.540221e-01 7.773459e-01 
          37           38           39           40           41           42 
5.618046e-01 3.039092e-02 1.666287e-01 4.472298e-01 3.760855e-03 3.228840e-02 
          43           44           45           46           47           48 
2.688257e-01 2.678672e-02 1.433095e-01 1.998281e-03 9.766915e-03 9.120030e-02 
          49           50           51           52           53           54 
5.349351e-03 1.367626e-01 5.097806e-01 8.420129e-02 3.297080e-03 1.935488e-02 
          55           56           57           58           59           60 
5.364101e-03 8.314481e-03 1.076808e-03 5.489316e-03 4.801636e-02 3.365189e-01 
          61           62           63           64           65           66 
2.820505e-02 3.721894e-03 3.089758e-02 2.820977e-01 3.680175e-03 7.049881e-03 
          67           68           69           70           71           72 
1.036950e-02 3.148857e-01 2.992207e-02 1.661940e-01 5.656592e-02 8.302960e-02 
          73           74           75           76           77           78 
3.152549e-02 5.506521e-01 3.333162e-02 5.689936e-01 2.048925e-03 1.083036e-03 
          79           80           81           82           83           84 
1.062461e-09 4.356581e-01 2.766306e-03 4.590089e-02 6.131897e-02 2.241244e-02 
          85           86           87           88           89           90 
5.869896e-02 1.020744e-02 1.309617e-02 3.606058e-02 1.740462e-01 3.364516e-02 
          91           92           93           94           95           96 
5.560361e-01 1.375341e-02 7.483582e-01 7.917115e-03 8.755249e-01 4.361772e-03 
          97           98           99          100          101          102 
3.830388e-02 5.440143e-02 2.102317e-02 7.155469e-03 3.536128e-01 3.576894e-02 
         103          104          105          106          107          108 
1.316752e-01 1.393994e-02 4.595547e-02 5.868746e-02 2.779603e-02 1.552787e-02 
         109          110          111          112          113          114 
1.729241e-02 5.661487e-02 2.301534e-02 6.009488e-03 1.740846e-01 2.521917e-02 
         115          116          117          118          119          120 
9.231171e-02 1.243927e-02 2.206241e-02 1.048723e-01 3.190140e-02 1.138178e-02 
         121          122          123          124          125          126 
6.373194e-03 1.038627e-01 8.645757e-02 3.531605e-01 9.760384e-01 8.857141e-02 
         127          128          129          130          131          132 
5.015444e-03 4.000831e-02 1.194930e-01 2.141315e-01 2.589531e-02 2.565643e-02 
         133          134          135          136          137          138 
2.163098e-03 1.516306e-03 1.034192e-03 5.359162e-05 9.838208e-04 7.160198e-04 
         139          140          141          142          143          144 
7.988756e-01 3.629382e-04 1.107131e-01 3.569777e-04 1.534985e-02 2.052402e-01 
         145          146          147          148          149          150 
9.241203e-03 2.263822e-01 6.592731e-01 3.214051e-01 1.566701e-02 1.056295e-01 
         151          152          153          154          

In [41]:
# create confusion matrix
confusionMatrix(predicted = pred_class, actual = churn_train$Attrition)

ERROR: Error in is.factor(data): argument "data" is missing, with no default


In [ ]:
# predict class
pred_class <- predict(cv_model3, churn_train)
# create confusion matrix
confusionMatrix(
data = relevel

In [ ]:
# predict class
pred_class <- predict(cv_model3, churn_train)
# create confusion matrix
confusionMatrix(
data = relevel(pred_class, ref = ”Yes”),
reference = relevel(churn_train$Attrition, ref = ”Yes”)
)

In [47]:
# Plot AUC

library(ROCR)
# Compute predicted probabilities
m1_prob <- predict(cv_model1, churn_train, type = "prob")$Yes
m3_prob <- predict(cv_model3, churn_train, type = "prob")$Yes

# Compute AUC metrics for cv_model1 and cv_model3
perf1 <- prediction(m1_prob, churn_train$Attrition) %>%
performance(measure = "tpr", x.measure = "fpr")
perf2 <- prediction(m3_prob, churn_train$Attrition) %>%
performance(measure = "tpr", x.measure = "fpr")

# Plot ROC curves for cv_model1 and cv_model3
plot(perf1, col = "black", lty = 2)
plot(perf2, add = TRUE, col = "blue")
legend(0.8, 0.2, legend = c("cv_model1", "cv_model3"),
col = c("black", "blue"), lty = 2:1, cex = 0.6)

Warning message:
"package 'ROCR' was built under R version 4.0.3"


ERROR: Error in dimnames(out)[[2]] <- modelFit$obsLevels: length of 'dimnames' [2] not equal to array extent
